In [1]:
import requests
import numpy as np
import pandas as pd
import json
from datetime import datetime
import decimal
import win32com.client
import pythoncom
from prefect import task, flow
from dotenv import load_dotenv
import os
from prefect.blocks.system import Secret
from postmarker.core import PostmarkClient
secret_block_un = Secret.load("lg-username")
secret_block_pw = Secret.load("lg-pw")
secret_block_postmark_token = Secret.load ("postmark-token")

ModuleNotFoundError: No module named 'dotenv'

In [2]:
# Task 1: Authentication
@task
def authenticate():
  GetTokenURL = "https://clmapi.landgorilla.com/api/token"
  USER = "dustinh@legacyg.com"
  PW = "Index@2043!"
  body = {'api_name': 'clm'}
  headers = {
    'USER': USER,
    'PASSWORD': PW
    }
  response = requests.get(GetTokenURL, headers=headers, json=body)
  json_data = response.json()
  if response.status_code == 200:
    json_data = response.json()
    api_token = json_data.get('token')
    

    if api_token:
        your_variable_name = api_token
        print(f"API Token: {api_token}")
    else:
        print("No API token found in the response JSON.")
  else:
     print("Request was not successful. Status code:", response.status_code)
  return json_data.get('token')

In [3]:
# Task 2: Get Report List
@task
def get_report(api_token):

    ReportURL = f"https://clmapi.landgorilla.com/api/clm/pipelineReport"

    headers = {
       'Authorization': f'Bearer {api_token}'
    }

    ReportResponse = requests.get(ReportURL, headers=headers, json=body)

    print(ReportResponse.text)

    ReportResponse.status_code

    return ReportResponse

In [4]:
# Task 3: Get latest Report ID Value
@task
def get_latest_id_value(ReportResponse):

    ReportData = ReportResponse.json()

    # Function to convert date string to a comparable format (e.g., MM/DD/YYYY)
    def convert_date(date_string):
    # Assuming the date format is MM/DD/YYYY
        return datetime.strptime(date_string, '%m/%d/%Y')

    # Get the dictionary with the most recent reportdate for a specific report name
    report_name = 'Capital Cash Status -  Combined'
    latest_report = max((data for data in ReportData if data['name'] == report_name), key=lambda x: int(x['id']), default=None)

    latest_id_value = latest_report.get('id')
    print(latest_id_value)

    return latest_id_value

In [ ]:
# Task 4: Get Watchlist Report
@task
def get_watchlist_report(latest_id_value, api_token):
    
    ActualReportURL = f"https://clmapi.landgorilla.com/api/clm/pipelineReport/{latest_id_value}"

    headers = {
        'Authorization': f'Bearer {api_token}'
    }

    ActualReportResponse = requests.get(ActualReportURL, headers=headers)

    print(ActualReportResponse.text)

    return ActualReportResponse

In [ ]:
# Task 5: Converting Watchlist Report Data to a Dataframe
@task
def convert_to_df(ActualReportResponse):

    ReportData = ActualReportResponse.json()
    data_list = ReportData ['data']['Current Report']['data']

    df = pd.DataFrame(data_list)

    return df

In [ ]:
# Task 6: Cleaning/Transforming Data
@task
def clean_df(df):

    #dropping rows that contain only null values and nothing else in order to get rid of the totals at the bottom
    df = df.dropna (subset = ['Loan Number'], how = 'all')

    #renaming column
    df.rename(columns={"Builder's Risk Insurance Expiration": "Builders Risk Insurance Expiration"}, inplace=True)

    #replacing all empty values with NaN values
    df.replace('', np.nan, inplace=True)

    #removing dollar signs, commas, and percent signs from numeric columns to convert them to numeric datatype

    df ['Collateral Holdback'] = df ['Collateral Holdback'].str.replace ("$", "")
    df ['Collateral Holdback'] = df ['Collateral Holdback'].str.replace (",", "")
    df ['Collateral Holdback'] = df ['Collateral Holdback'].str.replace ("%", "")

    df ['Appraised Value'] = df ['Appraised Value'].str.replace ("$", "")
    df ['Appraised Value'] = df ['Appraised Value'].str.replace (",", "")
    df ['As Permitted Property Value'] = df ['As Permitted Property Value'].str.replace ("$", "")
    df ['As Permitted Property Value'] = df ['As Permitted Property Value'].str.replace (",", "")
    df ['Current Loan Amount'] = df ['Current Loan Amount'].str.replace ("$", "")
    df ['Current Loan Amount'] = df ['Current Loan Amount'].str.replace (",", "")
    df ['Total Principal Loan Paydown'] = df ['Total Principal Loan Paydown'].str.replace ("$", "")
    df ['Total Principal Loan Paydown'] = df ['Total Principal Loan Paydown'].str.replace (",", "")
    df ['Total Interest Reserve Amount Disbursed'] = df ['Total Interest Reserve Amount Disbursed'].str.replace ("$", "")
    df ['Total Interest Reserve Amount Disbursed'] = df ['Total Interest Reserve Amount Disbursed'].str.replace (",", "")
    df ['Interest Reserve Balance'] = df ['Interest Reserve Balance'].str.replace ("$", "")
    df ['Interest Reserve Balance'] = df ['Interest Reserve Balance'].str.replace (",", "")
    df ['Loan amount Disbursed excluding Interest Reserve'] = df ['Loan amount Disbursed excluding Interest Reserve'].str.replace ("$", "")
    df ['Loan amount Disbursed excluding Interest Reserve'] = df ['Loan amount Disbursed excluding Interest Reserve'].str.replace (",", "")
    df ['Balance to finish Including Retainage'] = df ['Balance to finish Including Retainage'].str.replace ("$", "")
    df ['Balance to finish Including Retainage'] = df ['Balance to finish Including Retainage'].str.replace (",", "")
    df ['Loan Amount Disbursed'] = df ['Loan Amount Disbursed'].str.replace ("$", "")
    df ['Loan Amount Disbursed'] = df ['Loan Amount Disbursed'].str.replace (",", "")


    df ['LTV'] = df ['LTV'].str.replace ("%", "")
    df ['CLTV'] = df ['CLTV'].str.replace ("%", "")
    df ['LTC'] = df ['LTC'].str.replace ("%", "")
    df ['As Permitted LTV'] = df ['As Permitted LTV'].str.replace ("%", "")
    df ['As Permitted CLTV'] = df ['As Permitted CLTV'].str.replace ("%", "")
    df ['Current Interest Rate'] = df ['Current Interest Rate'].str.replace ("%", "")
    df ['% DISB'] = df ['% DISB'].str.replace ("%", "")


    #stripping spaces from numeric columns to change them to numeric

    df ['Appraised Value'] = df ['Appraised Value'].str.replace (" ", "")
    df ['Appraised Value'] = df ['Appraised Value'].str.strip()
    df ['As Permitted Property Value'] = df ['As Permitted Property Value'].str.replace (" ", "")
    df ['As Permitted Property Value'] = df ['As Permitted Property Value'].str.strip()
    df ['Current Loan Amount'] = df ['Current Loan Amount'].str.replace (" ", "")
    df ['Current Loan Amount'] = df ['Current Loan Amount'].str.strip()
    df ['Total Principal Loan Paydown'] = df ['Total Principal Loan Paydown'].str.replace (" ", "")
    df ['Total Principal Loan Paydown'] = df ['Total Principal Loan Paydown'].str.strip()
    df ['Total Interest Reserve Amount Disbursed'] = df ['Total Interest Reserve Amount Disbursed'].str.replace (" ", "")
    df ['Total Interest Reserve Amount Disbursed'] = df ['Total Interest Reserve Amount Disbursed'].str.strip()
    df ['Interest Reserve Balance'] = df ['Interest Reserve Balance'].str.replace (" ", "")
    df ['Interest Reserve Balance'] = df ['Interest Reserve Balance'].str.strip()
    df ['Loan amount Disbursed excluding Interest Reserve'] = df ['Loan amount Disbursed excluding Interest Reserve'].str.replace (" ", "")
    df ['Loan amount Disbursed excluding Interest Reserve'] = df ['Loan amount Disbursed excluding Interest Reserve'].str.strip()
    df ['Balance to finish Including Retainage'] = df ['Balance to finish Including Retainage'].str.replace (" ", "")
    df ['Balance to finish Including Retainage'] = df ['Balance to finish Including Retainage'].str.strip()
    df ['Loan Amount Disbursed'] = df ['Loan Amount Disbursed'].str.replace (" ", "")
    df ['Loan Amount Disbursed'] = df ['Loan Amount Disbursed'].str.strip()


    #changing datatypes to string
    df ['Loan Number'] = df ['Loan Number'].astype ('string')
    df ['Borrower Last Name'] = df ['Borrower Last Name'].astype ('string')
    df ['Property Address'] = df ['Property Address'].astype ('string')
    df ['Loan Program'] = df ['Loan Program'].astype ('string')
    df ['Loan Term in Months'] = df ['Loan Term in Months'].astype ('string')
    df ['Loan Originator'] = df ['Loan Originator'].astype ('string')
    df ['Total Units'] = df ['Total Units'].astype ('string')
    df ['County'] = df ['County'].astype ('string')
    df ['Property Address'] = df ['Property Address'].astype ('string')
    df ['City'] = df ['City'].astype ('string')
    df ['Investor'] = df ['Investor'].astype ('string')
    df ['Outside Equity'] = df ['Outside Equity'].astype ('string')
    df ['State'] = df ['State'].astype ('string')
    df ['Zip'] = df ['Zip'].astype ('string')
    df ['Exception'] = df ['Exception'].astype ('string')
    df ['Additional Collateral Property'] = df ['Additional Collateral Property'].astype ('string')
    df ['Funding Account'] = df ['Funding Account'].astype ('string')

    #changing datatypes to numeric

    df ['Appraised Value'] = df ['Appraised Value'].astype ('float64')
    df ['As Permitted Property Value'] = df ['As Permitted Property Value'].astype ('float64')
    df ['Current Loan Amount'] = df ['Current Loan Amount'].astype ('float64')
    df ['Total Principal Loan Paydown'] = df ['Total Principal Loan Paydown'].astype ('float64')
    df ['Total Interest Reserve Amount Disbursed'] = df ['Total Interest Reserve Amount Disbursed'].astype ('float64')
    df ['Interest Reserve Balance'] = df ['Interest Reserve Balance'].astype ('float64')
    df ['Loan amount Disbursed excluding Interest Reserve'] = df ['Loan amount Disbursed excluding Interest Reserve'].astype ('float64')
    df ['Balance to finish Including Retainage'] = df ['Balance to finish Including Retainage'].astype ('float64')
    df ['Loan Amount Disbursed'] = df ['Loan Amount Disbursed'].astype ('float64')

    df ['LTV'] = df ['LTV'].astype ('float64')
    df ['CLTV'] = df ['CLTV'].astype ('float64')
    df ['LTC'] = df ['LTC'].astype ('float64')
    df ['As Permitted LTV'] = df ['As Permitted LTV'].astype ('float64')
    df ['As Permitted CLTV'] = df ['As Permitted CLTV'].astype ('float64')
    df ['Current Interest Rate'] = df ['Current Interest Rate'].astype ('float64')
    df ['% DISB'] = df ['% DISB'].astype ('float64')

    #changing datatypes to datetime
    df ['Loan Funded Date'] = pd.to_datetime (df ['Loan Funded Date'])
    df ['Original Loan Due Date'] = pd.to_datetime (df ['Original Loan Due Date'])
    df ['Current Loan Due Date'] = pd.to_datetime (df ['Current Loan Due Date'])
    df ['Builders Risk Insurance Expiration'] = pd.to_datetime (df ['Builders Risk Insurance Expiration'])

In [ ]:
@flow (log_prints= True)
def Watchlist_Report_LG() -> None:
    api_token = authenticate()
    ReportResponse = get_report(api_token)
    latest_id_value = get_latest_id_value(ReportResponse)
    ActualReportResponse = get_watchlist_report(latest_id_value, api_token)
    df = convert_to_df(ActualReportResponse)
    clean_df(df)

if __name__ == "__main__":
    Watchlist_Report_LG()